In [1]:
from keras.datasets import cifar10

import numpy as np
import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

random_state = 42

Using TensorFlow backend.


In [0]:
(cifar_X_1, cifar_y_1), (cifar_X_2, cifar_y_2) = cifar10.load_data()

cifar_X = np.r_[cifar_X_1, cifar_X_2]
cifar_y = np.r_[cifar_y_1, cifar_y_2]

cifar_X = cifar_X.astype('float32') / 255
cifar_y = np.eye(10)[cifar_y.astype('int32').flatten()]

train_X, test_X, train_y, test_y = train_test_split(
    cifar_X, cifar_y, test_size=10000, random_state=random_state)
train_X, valid_X, train_y, valid_y = train_test_split(
    train_X, train_y, test_size=10000, random_state=random_state)

In [3]:
cifar_X_1.shape

(50000, 32, 32, 3)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Input, Lambda
from keras.callbacks import EarlyStopping

In [0]:
model = Sequential()

In [12]:
model = Sequential()

from keras.layers.core import Dropout

# 32x32x3 -> 30x30x64
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x64 -> 14x14x64
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x64 -> 12x12x128
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # 12x12x128 -> 5x5x128
model.add(Dropout(0.2))

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->120
# 120 ->84
model.add(Dropout(0.3))
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  
model.add(Dense(10, activation='softmax'))  # 84 ->10

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [14]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=train_X, y=train_y, batch_size=256, epochs=10, verbose=1,
          validation_data=(test_X, test_y), callbacks=[early_stopping])

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 6s 152us/step - loss: 1.8799 - acc: 0.3015 - val_loss: 1.4982 - val_acc: 0.4533
Epoch 2/10
40000/40000 [==============================] - 5s 136us/step - loss: 1.4452 - acc: 0.4729 - val_loss: 1.2430 - val_acc: 0.5587
Epoch 3/10
40000/40000 [==============================] - 5s 137us/step - loss: 1.2790 - acc: 0.5397 - val_loss: 1.1346 - val_acc: 0.5981
Epoch 4/10
40000/40000 [==============================] - 5s 137us/step - loss: 1.1318 - acc: 0.5998 - val_loss: 0.9772 - val_acc: 0.6653
Epoch 5/10
40000/40000 [==============================] - 6s 138us/step - loss: 1.0140 - acc: 0.6420 - val_loss: 0.8884 - val_acc: 0.6917
Epoch 6/10
40000/40000 [==============================] - 6s 140us/step - loss: 0.9228 - acc: 0.6775 - val_loss: 0.8442 - val_acc: 0.7090
Epoch 7/10
40000/40000 [==============================] - 6s 139us/step - loss: 0.8506 - acc: 0.7022 - val_loss: 0.7942 - 

In [9]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9020908591270447
Test accuracy: 0.6895


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
__________